In [1]:
import openai
import re

In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

True

In [3]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    input="Write a one-sentence bedtime story about a unicorn.",
)

print(response.output_text)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-*********************************************************************************************************************************************************2oA\\. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [74]:
class Agent:
    """This is a agent class for creating agents."""

    def __init__(self, system):
        self.system = system
        self.messages = [{"role": "system", "content": system}]

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        response = client.responses.create(
            model="gpt-4o-mini-2024-07-18",
            input=self.messages,
        )

        return response.output_text

In [75]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [104]:
def calculate(what):
    return eval(what)


def average_dog_weight(name):
    name = name.strip()
    if name == "Scottish Terrier":
        print(f"inside {name}")
        return "Scottish Terriers average 20 lbs"
    elif name == "Border Collie":
        print(f"inside {name}")
        return "a Border Collies average weight is 37 lbs"
    elif name == "Toy Poodle":
        print(f"inside {name}")
        return "a toy poodles average weight is 7 lbs"
    else:
        print(f"inside else")
        return "An average dog weights 50 lbs"


known_actions = {"calculate": calculate, "average_dog_weight": average_dog_weight}

In [77]:
bot = Agent(system=prompt)

In [ ]:
bot.messages

In [79]:
result = bot("How much does a toy poodle weigh?")

In [84]:
result = average_dog_weight("Toy Poodle")

In [ ]:
print(result)

In [ ]:
next_prompt = "Observation: {}".format(result)
next_prompt

In [ ]:
bot(next_prompt)

In [89]:
action_re = re.compile(
    "^Action: (\w+): (.*)$"
)  # python regular expression to selection action

In [103]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input.strip()))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [ ]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

In [ ]:
question = """I have 2 dogs, a Border Collie and a Scottish Terrier. \
What is their combined weight"""
query(question)